# XGBoost

## Importing the dataset

In [1]:
dataset = read.csv('Churn_Modelling.csv')
dataset = dataset[4:14]
head(dataset)

CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,France,Female,42,2,0.00,1,1,1,101348.88,1
608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
502,France,Female,42,8,159660.80,3,1,0,113931.57,1
699,France,Female,39,1,0.00,2,0,0,93826.63,0
850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1


## Encoding the categorical variables as factors

In [2]:
dataset$Geography = as.numeric(factor(dataset$Geography,
                                      levels=c('France', 'Spain', 'Germany'),
                                      labels=c(1, 2, 3)))
dataset$Gender = as.numeric(factor(dataset$Gender,
                                   levels=c('Female', 'Male'),
                                   labels=c(1, 2)))

In [3]:
head(dataset)

CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,1,1,42,2,0.00,1,1,1,101348.88,1
608,2,1,41,1,83807.86,1,0,1,112542.58,0
502,1,1,42,8,159660.80,3,1,0,113931.57,1
699,1,1,39,1,0.00,2,0,0,93826.63,0
850,2,1,43,2,125510.82,1,1,1,79084.10,0
645,2,2,44,8,113755.78,2,1,0,149756.71,1


## Splitting the dataset into the Training set and Test set

In [4]:
library(caTools)
set.seed(123)

split = sample.split(dataset$Exited, SplitRatio=0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

Warning message:
"package 'caTools' was built under R version 3.6.3"

## Fitting XGBoost to the Training set

In [5]:
# install.packages('xgboost')

In [7]:
library(xgboost)

classifier = xgboost(data=as.matrix(training_set[-11]), label=training_set$Exited, nrounds=10)

[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316158 
[6]	train-rmse:0.310497 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300683 
[10]	train-rmse:0.298272 


## Predicting the Test set results

In [10]:
y_pred = predict(classifier, newdata=as.matrix(test_set[-11]))
y_pred = (y_pred >= 0.5)
head(data.frame(y_test=test_set[11], y_pred=as.integer(y_pred)))

,Exited,y_pred
2,0,0
4,0,0
8,1,1
12,0,0
13,0,0
14,0,0


## Making the Confusion Matrix

In [11]:
cm = table(test_set[, 11], y_pred)
cm

   y_pred
    FALSE TRUE
  0  1541   52
  1   211  196

## Applying k-Fold Cross Validation

In [15]:
library(caret)

folds = createFolds(dataset$Exited, k=10)
cv = lapply(folds, function(x) {
                                training_fold=dataset[-x, ]
                                test_fold=dataset[x, ]
                                classifier=xgboost(data=as.matrix(training_fold[-11]), label=training_fold$Exited, nrounds=10, verbose=0)
                                y_pred=predict(classifier, newdata=as.matrix(test_fold[-11]))
                                y_pred=(y_pred >= 0.5)
                                cm=table(test_fold[, 11], y_pred)
                                accuracy=(cm[1,1] + cm[2,2]) / (cm[1,1] + cm[2,2] + cm[1,2] + cm[2,1])
                                return(accuracy)
                                })
accuracy = mean(as.numeric(cv))

In [16]:
accuracy

[1] 0.8629